# 0.1. imports

In [1]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
# configuration
import yaml
# lca
import ecospold2matrix as e2m
import pymrio
#import brightway2 as bw
# type hints
from ecospold2matrix import ecospold2matrix
from pymrio import IOSystem
# data science
import pandas as pd
# deep copy
import copy

# 0.2. config

In [2]:
with open('../config.yaml', 'r') as filestream:
    config = yaml.load(filestream, Loader = yaml.FullLoader)

# 0.3. file paths

## 0.3.1. directories

In [3]:
%%capture
print(path_dir_data := Path(Path.home(), config['path_dir_data']))
print(path_dir_data_raw := Path(path_dir_data, config['path_dir_data_raw']))
# exiobase
print(path_dir_exiobase := Path(path_dir_data_raw, 'IOT_2011_pxp'))

## 0.3.2. files

In [19]:
%% capture
print(path_file_exiobase3 := Path(path_dir_data_raw, 'IOT_2011_pxp.zip'))

UsageError: Cell magic `%%` not found.


In [4]:
%%capture
print(path_file_impacts_S := Path(path_dir_exiobase, 'impacts', 'S.txt'))
print(path_file_impacts_S_Y := Path(path_dir_exiobase, 'impacts', 'S_Y.txt'))
print(path_file_impacts_M := Path(path_dir_exiobase, 'impacts', 'M.txt'))
print(path_file_impacts_F := Path(path_dir_exiobase, 'impacts', 'F.txt'))
print(path_file_impacts_F_Y := Path(path_dir_exiobase, 'impacts', 'F_Y.txt'))

In [5]:
%%capture
print(path_file_satellite_S := Path(path_dir_exiobase, 'satellite', 'S.txt'))
print(path_file_satellite_S_Y := Path(path_dir_exiobase, 'satellite', 'S_Y.txt'))
print(path_file_satellite_M := Path(path_dir_exiobase, 'satellite', 'M.txt'))
print(path_file_satellite_F := Path(path_dir_exiobase, 'satellite', 'F.txt'))
print(path_file_satellite_F_Y := Path(path_dir_exiobase, 'satellite', 'F_Y.txt'))

In [6]:
%%capture
print(path_file_C_IWP := Path(path_dir_data_raw, 'exiobase_characterization', 'C_exio_IW_1.28_1.46.csv'))

# 1. load files

In [7]:
def read_csv(filepath: Path) -> pd.DataFrame:
    df = pd.read_csv(
        filepath_or_buffer = filepath,
        sep = ',',
        engine = 'python',
        encoding = 'utf-8',
        index_col = None
    )
    return df

In [8]:
def read_ecoinvent_txt(filepath: Path) -> pd.DataFrame:
    df = pd.read_csv(
        filepath_or_buffer = filepath,
        sep = '\t',
        engine = 'python',
        encoding = 'utf-8',
        index_col = None
    )
    return df

In [9]:
impacts_S = read_ecoinvent_txt(path_file_impacts_S)
impacts_S_Y = read_ecoinvent_txt(path_file_impacts_S_Y)
impacts_M = read_ecoinvent_txt(path_file_impacts_M)
impacts_F = read_ecoinvent_txt(path_file_impacts_F)
impacts_F_Y = read_ecoinvent_txt(path_file_impacts_F_Y)

In [10]:
satellite_S = read_ecoinvent_txt(path_file_satellite_S)
satellite_S_Y = read_ecoinvent_txt(path_file_satellite_S_Y)
satellite_M = read_ecoinvent_txt(path_file_satellite_M)
satellite_F = read_ecoinvent_txt(path_file_satellite_F)
satellite_F_Y = read_ecoinvent_txt(path_file_satellite_F_Y)

In [11]:
C_IWP = read_csv(path_file_C_IWP)

# 2. investigation
#### 2.1. `C_IWP`...up-to-date characterization factors for Exiobase (https://zenodo.org/record/4739143)

In [12]:
C_IWP

,Unnamed: 0,Taxes less subsidies on products purchased: Total,Other net taxes on production,"Compensation of employees; wages, salaries, & employers' social contributions: Low-skilled","Compensation of employees; wages, salaries, & employers' social contributions: Medium-skilled","Compensation of employees; wages, salaries, & employers' social contributions: High-skilled",Operating surplus: Consumption of fixed capital,Operating surplus: Rents on land,Operating surplus: Royalties on resources,Operating surplus: Remaining net operating surplus,...,Water Withdrawal Blue - Domestic - domestic Water Withdrawal Blue,Energy Carrier Net Total,Energy Carrier Net NENE,Energy Carrier Net NTRA,Energy Carrier Net TAVI,Energy Carrier Net TMAR,Energy Carrier Net TOTH,Energy Carrier Net TRAI,Energy Carrier Net TROA,Energy Carrier Net LOSS
0,"Climate change, ecosystem quality, long term (...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Climate change, ecosystem quality, short term ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Climate change, human health, long term (DALY)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Climate change, human health, short term (DALY)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Climate change, long term (kg CO2 eq (long))",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"Climate change, short term (kg CO2 eq (short))",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Freshwater acidification (PDF.m2.yr),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Freshwater acidification (kg SO2 eq),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Freshwater ecotoxicity (CTUe),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"Freshwater ecotoxicity, long term (PDF.m2.yr)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
len(C_IWP.columns.tolist()[1:])

1113

In [16]:
C_IWP.columns.tolist()[-100:-1]

['Water Consumption Blue - Electricity - once-through - Electricity by coal',
 'Water Consumption Blue - Electricity - once-through - Electricity by gas',
 'Water Consumption Blue - Electricity - once-through - Electricity by nuclear',
 'Water Consumption Blue - Electricity - once-through - Electricity by hydro',
 'Water Consumption Blue - Electricity - once-through - Electricity by wind',
 'Water Consumption Blue - Electricity - once-through - Electricity by petroleum and other oil derivatives',
 'Water Consumption Blue - Electricity - once-through - Electricity by biomass and waste',
 'Water Consumption Blue - Electricity - once-through - Electricity by solar photovoltaic',
 'Water Consumption Blue - Electricity - once-through - Electricity by solar thermal',
 'Water Consumption Blue - Electricity - once-through - Electricity by tide, wave, ocean',
 'Water Consumption Blue - Electricity - once-through - Electricity by Geothermal',
 'Water Consumption Blue - Electricity - once-through

In [15]:
C_IWP['Unnamed: 0'].tolist()[-0:-10]

['Climate change, ecosystem quality, long term (PDF.m2.yr)',
 'Climate change, ecosystem quality, short term (PDF.m2.yr)',
 'Climate change, human health, long term (DALY)',
 'Climate change, human health, short term (DALY)',
 'Climate change, long term (kg CO2 eq (long))',
 'Climate change, short term (kg CO2 eq (short))',
 'Freshwater acidification (PDF.m2.yr)',
 'Freshwater acidification (kg SO2 eq)',
 'Freshwater ecotoxicity (CTUe)',
 'Freshwater ecotoxicity, long term (PDF.m2.yr)',
 'Freshwater ecotoxicity, short term (PDF.m2.yr)',
 'Freshwater eutrophication (PDF.m2.yr)',
 'Freshwater eutrophication (kg PO4 P-lim eq)',
 'Human toxicity cancer (CTUh)',
 'Human toxicity cancer, long term (DALY)',
 'Human toxicity cancer, short term (DALY)',
 'Human toxicity non cancer (CTUh)',
 'Human toxicity non-cancer, long term (DALY)',
 'Human toxicity non-cancer, short term (DALY)',
 'Marine acidification, long term (PDF.m2.yr)',
 'Marine acidification, short term (PDF.m2.yr)',
 'Marine eutro

#### 2.2. satellite accounts
##### 2.2.1. `F`...factors of production, stressors, impacts (https://zenodo.org/record/5589597)

In [16]:
satellite_F.iloc[0:10]

,region,AT,AT.1,AT.2,AT.3,AT.4,AT.5,AT.6,AT.7,AT.8,...,WM.190,WM.191,WM.192,WM.193,WM.194,WM.195,WM.196,WM.197,WM.198,WM.199
0,sector,Paddy rice,Wheat,Cereal grains nec,"Vegetables, fruit, nuts",Oil seeds,"Sugar cane, sugar beet",Plant-based fibers,Crops nec,Cattle,...,Paper for treatment: landfill,Plastic waste for treatment: landfill,Inert/metal/hazardous waste for treatment: lan...,Textiles waste for treatment: landfill,Wood waste for treatment: landfill,Membership organisation services n.e.c. (91),"Recreational, cultural and sporting services (92)",Other services (93),Private households with employed persons (95),Extra-territorial organizations and bodies
1,stressor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Taxes less subsidies on products purchased: Total,0,6.05377029467,11.8895027484,10.8862121456,1.31469542317,0.930543279762,0.000469976141195,4.95597808715,33.0055553199,...,42.3287384561,25.1483604814,45.0760630585,15.5015898912,18.0056255843,884.488801296,2580.21495989,1734.06131174,59.4091731566,0
3,Other net taxes on production,0,-12.2204612994,-26.9106680619,-39.7897202306,-5.83278067825,-2.54752318323,-0.00245025353334,-10.751813318,-217.488955606,...,17.1854536392,7.83256660304,18.7223759903,3.29474490043,3.81290766204,64.8498290536,840.852331491,434.272999966,24.3224145549,0
4,"Compensation of employees; wages, salaries, & ...",0,1.06593266337,2.1668770901,3.66573180869,0.249154144389,0.285979148307,0.000327544440266,0.613817758809,1.59354288038,...,25.2329768629,14.9107578066,25.3482843527,7.72738735479,10.0342276581,732.898440963,1168.28846027,633.149175173,1499.78730169,0
5,"Compensation of employees; wages, salaries, & ...",0,14.6680165556,29.8178206974,50.4431627872,3.42854407263,3.93528318074,0.00450725213483,8.44658331413,21.9283207606,...,158.907605972,92.5310272865,161.332596326,46.7956536741,61.6992991628,1660.63264764,3059.71275677,1756.60066748,1219.07056689,0
6,"Compensation of employees; wages, salaries, & ...",0,2.33657619605,4.74989987877,8.03546225728,0.546157991901,0.626880189965,0.000717993329764,1.34551835518,3.49312343044,...,89.3970218133,50.6658036003,96.0202909986,24.959671725,34.251517641,1568.23474579,2861.55621154,1656.9114861,157.211297569,0
7,Operating surplus: Consumption of fixed capital,0,80.7800918946,205.667887936,335.172366939,38.0999682893,23.6016114067,0.431541614059,42.3523520351,53.893556796,...,76.8447802028,45.3824385612,79.1407851573,13.8351706257,18.7169738516,157.337606507,1456.61515879,997.785177033,99.9973296768,0
8,Operating surplus: Rents on land,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Operating surplus: Royalties on resources,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# number of meaningful rows
len(satellite_F['region'].tolist()[2:])

1113

In [18]:
satellite_F['region'].tolist()[2:10]

['Taxes less subsidies on products purchased: Total',
 'Other net taxes on production',
 "Compensation of employees; wages, salaries, & employers' social contributions: Low-skilled",
 "Compensation of employees; wages, salaries, & employers' social contributions: Medium-skilled",
 "Compensation of employees; wages, salaries, & employers' social contributions: High-skilled",
 'Operating surplus: Consumption of fixed capital',
 'Operating surplus: Rents on land',
 'Operating surplus: Royalties on resources']

##### 2.2.2. `S`...characterization factors (https://zenodo.org/record/5589597)

In [19]:
satellite_S.iloc[0:10]

,region,AT,AT.1,AT.2,AT.3,AT.4,AT.5,AT.6,AT.7,AT.8,...,WM.190,WM.191,WM.192,WM.193,WM.194,WM.195,WM.196,WM.197,WM.198,WM.199
0,sector,Paddy rice,Wheat,Cereal grains nec,"Vegetables, fruit, nuts",Oil seeds,"Sugar cane, sugar beet",Plant-based fibers,Crops nec,Cattle,...,Paper for treatment: landfill,Plastic waste for treatment: landfill,Inert/metal/hazardous waste for treatment: lan...,Textiles waste for treatment: landfill,Wood waste for treatment: landfill,Membership organisation services n.e.c. (91),"Recreational, cultural and sporting services (92)",Other services (93),Private households with employed persons (95),Extra-territorial organizations and bodies
1,stressor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Taxes less subsidies on products purchased: Total,0,0.0179262100139,0.0151155752328,0.00746719686965,0.00840696939904,0.00933756964791,0.000304179982038,0.0267010106629,0.0953511185276,...,0.0334854671528,0.0341894760221,0.034458421362,0.0501446526717,0.04517923913,0.0755131648846,0.0840707829548,0.0883456418215,0.0129284560354,0
3,Other net taxes on production,0,-0.0361867968319,-0.0342125517157,-0.0272930262956,-0.0372983793882,-0.0255632120186,-0.00158586364377,-0.0579268667055,-0.628312869861,...,0.0135950884513,0.0106484614878,0.0143123306917,0.0106578641181,0.00956724698272,0.00553654927783,0.0273973738455,0.0221250117559,0.00529297498248,0
4,"Compensation of employees; wages, salaries, & ...",0,0.00315640201957,0.00275483292857,0.00251444126943,0.0015932445117,0.00286966793834,0.000211994723186,0.00330702723758,0.00460365216076,...,0.0199613323886,0.0202713412224,0.0193775100078,0.0249966072955,0.0251776184462,0.0625711492729,0.038066179407,0.032257204443,0.326379465696,0
5,"Compensation of employees; wages, salaries, & ...",0,0.0434344107002,0.0379085249877,0.0346005591496,0.0219242149884,0.0394887390877,0.002917203137,0.0455071243597,0.0633496358928,...,0.12570881189,0.125796961637,0.123330792585,0.151374911639,0.154814248357,0.141776387389,0.0996941925682,0.08949395984,0.265290684766,0
6,"Compensation of employees; wages, salaries, & ...",0,0.00691898660916,0.00603872764784,0.0055117774494,0.0034924693918,0.00629045156953,0.0004647027349,0.00724916440652,0.0100914292463,...,0.0707202989429,0.0688807239987,0.0734027646167,0.0807397227147,0.0859430014703,0.133887923468,0.0932376855855,0.0844150709609,0.0342118774071,0
7,Operating surplus: Consumption of fixed capital,0,0.239203144777,0.261473376882,0.229905316534,0.243634580198,0.236831209366,0.279304221868,0.22817909672,0.155695332865,...,0.0607904571977,0.0616979304144,0.0604992170308,0.04475410784,0.0469641353736,0.0134326863215,0.0474606878757,0.050834403183,0.0217611357252,0
8,Operating surplus: Rents on land,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Operating surplus: Royalties on resources,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# number of meaningful rows
len(satellite_S['region'].tolist()[2:])

1113

In [21]:
satellite_S['region'].tolist()[2:10]

['Taxes less subsidies on products purchased: Total',
 'Other net taxes on production',
 "Compensation of employees; wages, salaries, & employers' social contributions: Low-skilled",
 "Compensation of employees; wages, salaries, & employers' social contributions: Medium-skilled",
 "Compensation of employees; wages, salaries, & employers' social contributions: High-skilled",
 'Operating surplus: Consumption of fixed capital',
 'Operating surplus: Rents on land',
 'Operating surplus: Royalties on resources']

#### 2.3. impacts
##### 2.3.1. `F`...factors of production, stressors, impacts (https://zenodo.org/record/5589597)

In [22]:
impacts_F.iloc[10:20]

,region,AT,AT.1,AT.2,AT.3,AT.4,AT.5,AT.6,AT.7,AT.8,...,WM.190,WM.191,WM.192,WM.193,WM.194,WM.195,WM.196,WM.197,WM.198,WM.199
10,Respiratory effects on humans caused by organi...,0,0.531831905426,1.28692926827,0.427476795439,0.141828891826,0.090486571233,6.37015075904e-06,0.142779730382,0.448295013177,...,0.182593685632,0.155264359942,0.17347542763,0.0844667293525,0.100895994989,0.535105749915,1.20974417724,3.93514274329,0.0350348252911,0
11,Respiratory effects on humans caused by inorga...,0,993060.301927,1253894.97996,536717.08762,153378.6378,70939.240033,34.4850392872,109015.708649,888579.346808,...,38317965.5794,25076199.888,58099330.0262,18451629.2558,20639350.4651,80754222.7628,94466505.8864,156566741.684,562868.406186,0
12,Damages to human health caused by climate chan...,9.51138942895e-12,103.527648623,149.015585124,47.8492422359,34.889471455,17.0689359166,0.104446275838,12.8301240137,400.32111911,...,5646.52433295,247.063105301,69.444047218,531.867900129,165.604520594,100.179340652,654.461251351,1216.14860719,13.2559070245,0
13,Damage to Ecosystem Quality caused by ecotoxic...,0,992940.198051,1253692.62925,536648.458073,153354.230623,70930.6810247,34.4830629582,109000.296075,888457.397712,...,38318247.0791,25076317.0935,58099920.6916,18451746.5014,20639473.1395,80755013.2526,94467140.1656,156567959.616,562870.129002,0
14,Damage to Ecosystem Quality caused by the comb...,0,85996055.436,143029302.393,37390185.0982,37856563.0934,18065484.989,125757.320374,11221580.8745,419783256.764,...,17907226.8527,17153659.7587,16624653.7317,10587270.2926,11531498.2988,15778542.2103,254347064.406,93422435.6844,878234.811476,0
15,Carcinogenic effects on humans (E.E) | ECOINDI...,0,6.9314373997,14.3046937388,4.20033136288,1.72695038614,0.510191775441,1.64388585715e-05,1.00456778232,6.86738333092,...,72.0100525621,55.9897447645,103.017228091,25.9197942267,29.2536284986,97.8589524119,205.747338873,281.20586558,1.15046838218,0
16,Respiratory effects on humans caused by organi...,2.76694965206e-14,120.635818758,203.637822785,69.0571436452,24.5490330689,8.64950708812,0.00198274953583,15.5553686121,123.19345121,...,304.294828883,235.249664002,297.441393459,172.975741906,200.405846136,519.418182735,625.691444468,977.038019452,3.87644145962,0
17,Respiratory effects on humans caused by inorga...,0,564.593591442,915.451407411,255.430863002,223.05712293,105.235244736,0.689402447691,72.3508389726,2385.22836951,...,213.134059209,197.553399803,203.382560593,114.979528133,129.793192884,196.632844762,1797.59533717,1516.09828325,11.176968255,0
18,Damages to human health caused by climate chan...,9.51138942895e-12,103.527648623,149.015585124,47.8492422359,34.889471455,17.0689359166,0.104446275838,12.8301240137,400.32111911,...,5646.52433295,247.063105301,69.444047218,531.867900129,165.604520594,100.179340652,654.461251351,1216.14860719,13.2559070245,0
19,Damage to Ecosystem Quality caused by ecotoxic...,0,992940.198051,1253692.62925,536648.458073,153354.230623,70930.6810247,34.4830629582,109000.296075,888457.397712,...,38318247.0791,25076317.0935,58099920.6916,18451746.5014,20639473.1395,80755013.2526,94467140.1656,156567959.616,562870.129002,0


In [23]:
# number of meaningful rows
len(satellite_F['region'].tolist()[2:])

1113

In [24]:
impacts_F['region'].tolist()[2:10]

['Value Added',
 'Employment',
 'Employment hour',
 'GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)',
 'Human toxicity (USEtox) | USEtox2008 | CTUh (Rosenbaum et al., 2008)',
 'Fresh water Ecotoxicity (USEtox) | USEtox2008 | CTUe (Rosenbaum et al., 2008)',
 'EPS | Damage Approach | EPS (Steen, 1999))',
 'Carcinogenic effects on humans (H.A) | ECOINDICATOR 99 (H.A) | Carcinogenic effects on humans (H.A)']

# 3. `pymrio` investigation

In [23]:
exiobase_pymrio: pymrio.core.mriosystem.IOSystem = pymrio.parse_exiobase3(path = path_file_exiobase3)

In [28]:
type(exiobase_pymrio.impacts)

pymrio.core.mriosystem.Extension

## 3.1. satellite

In [26]:
exiobase_pymrio.satellite.S

region                                                     AT            \
sector                                             Paddy rice     Wheat   
stressor                                                                  
Taxes less subsidies on products purchased: Total           0  0.017926   
Other net taxes on production                               0 -0.036187   
Compensation of employees; wages, salaries, & e...          0  0.003156   
Compensation of employees; wages, salaries, & e...          0  0.043434   
Compensation of employees; wages, salaries, & e...          0  0.006919   
...                                                       ...       ...   
Energy Carrier Net TMAR                                     0  0.000000   
Energy Carrier Net TOTH                                     0  0.000000   
Energy Carrier Net TRAI                                     0  0.000000   
Energy Carrier Net TROA                                     0  4.046195   
Energy Carrier Net LOSS                                     0  0.000000   

region                                                                \
sector                                             Cereal grains nec   
stressor                                                               
Taxes less subsidies on products purchased: Total           0.015116   
Other net taxes on production                              -0.034213   
Compensation of employees; wages, salaries, & e...          0.002755   
Compensation of employees; wages, salaries, & e...          0.037909   
Compensation of employees; wages, salaries, & e...          0.006039   
...                                                              ...   
Energy Carrier Net TMAR                                     0.000000   
Energy Carrier Net TOTH                                     0.000000   
Energy Carrier Net TRAI                                     0.000000   
Energy Carrier Net TROA                                     0.296270   
Energy Carrier Net LOSS                                     0.000000   

region                                                                      \
sector                                             Vegetables, fruit, nuts   
stressor                                                                     
Taxes less subsidies on products purchased: Total                 0.007467   
Other net taxes on production                                    -0.027293   
Compensation of employees; wages, salaries, & e...                0.002514   
Compensation of employees; wages, salaries, & e...                0.034601   
Compensation of employees; wages, salaries, & e...                0.005512   
...                                                                    ...   
Energy Carrier Net TMAR                                           0.000000   
Energy Carrier Net TOTH                                           0.000000   
Energy Carrier Net TRAI                                           0.000000   
Energy Carrier Net TROA                                           0.445856   
Energy Carrier Net LOSS                                           0.000000   

region                                                        \
sector                                             Oil seeds   
stressor                                                       
Taxes less subsidies on products purchased: Total   0.008407   
Other net taxes on production                      -0.037298   
Compensation of employees; wages, salaries, & e...  0.001593   
Compensation of employees; wages, salaries, & e...  0.021924   
Compensation of employees; wages, salaries, & e...  0.003492   
...                                                      ...   
Energy Carrier Net TMAR                             0.000000   
Energy Carrier Net TOTH                             0.000000   
Energy Carrier Net TRAI                             0.000000   
Energy Carrier Net TROA                             0.230765   
Energy Carrier 

In [34]:
[*exiobase_pymrio.satellite.get_index()][-20:-1]

['Water Withdrawal Blue - Electricity - once-through - Electricity by nuclear',
 'Water Withdrawal Blue - Electricity - once-through - Electricity by hydro',
 'Water Withdrawal Blue - Electricity - once-through - Electricity by wind',
 'Water Withdrawal Blue - Electricity - once-through - Electricity by petroleum and other oil derivatives',
 'Water Withdrawal Blue - Electricity - once-through - Electricity by biomass and waste',
 'Water Withdrawal Blue - Electricity - once-through - Electricity by solar photovoltaic',
 'Water Withdrawal Blue - Electricity - once-through - Electricity by solar thermal',
 'Water Withdrawal Blue - Electricity - once-through - Electricity by tide, wave, ocean',
 'Water Withdrawal Blue - Electricity - once-through - Electricity by Geothermal',
 'Water Withdrawal Blue - Electricity - once-through - Electricity nec',
 'Water Withdrawal Blue - Domestic - domestic Water Withdrawal Blue',
 'Energy Carrier Net Total',
 'Energy Carrier Net NENE',
 'Energy Carrier 

## 3.2. impacts

In [27]:
exiobase_pymrio.impacts.S

region                                                     AT                \
sector                                             Paddy rice         Wheat   
impact                                                                        
Value Added                                                 0  6.759882e-01   
Employment                                                  0  2.841612e-02   
Employment hour                                             0  7.349490e+04   
GHG emissions (GWP100) | Problem oriented appro...          0  1.581915e+06   
Human toxicity (USEtox) | USEtox2008 | CTUh (Ro...          0  6.203164e-03   
...                                                       ...           ...   
Unused Domestic Extraction - Oil and Gas                    0  0.000000e+00   
Unused Domestic Extraction - Non-metalic Minerals           0  0.000000e+00   
Unused Domestic Extraction - Iron Ore                       0  0.000000e+00   
Unused Domestic Extraction - Non-ferous metal ores          0  0.000000e+00   
Land use Crop, Forest, Pasture                              0  9.463800e+00   

region                                                                \
sector                                             Cereal grains nec   
impact                                                                 
Value Added                                             7.547951e-01   
Employment                                              2.480092e-02   
Employment hour                                         6.414461e+04   
GHG emissions (GWP100) | Problem oriented appro...      1.052707e+06   
Human toxicity (USEtox) | USEtox2008 | CTUh (Ro...      2.756244e-03   
...                                                              ...   
Unused Domestic Extraction - Oil and Gas                0.000000e+00   
Unused Domestic Extraction - Non-metalic Minerals       0.000000e+00   
Unused Domestic Extraction - Iron Ore                   0.000000e+00   
Unused Domestic Extraction - Non-ferous metal ores      0.000000e+00   
Land use Crop, Forest, Pasture                          6.715495e+00   

region                                                                      \
sector                                             Vegetables, fruit, nuts   
impact                                                                       
Value Added                                                       0.678709   
Employment                                                        0.022637   
Employment hour                                               58547.236579   
GHG emissions (GWP100) | Problem oriented appro...           235793.434977   
Human toxicity (USEtox) | USEtox2008 | CTUh (Ro...                0.000750   
...                                                                    ...   
Unused Domestic Extraction - Oil and Gas                          0.000000   
Unused Domestic Extraction - Non-metalic Minerals                 0.000000   
Unused Domestic Extraction - Iron Ore                             0.000000   
Unused Domestic Extraction - Non-ferous metal ores                0.000000   
Land use Crop, Forest, Pasture                                    1.007479   

region                                                            \
sector                                                 Oil seeds   
impact                                                             
Value Added                                         8.030163e-01   
Employment                                          1.434349e-02   
Employment hour                                     3.709773e+04   
GHG emissions (GWP100) | Problem oriented appro...  1.079850e+06   
Human toxicity (USEtox) | USEtox2008 | CTUh (Ro...  1.708089e-03   
...                                                          ...   
Unused Domestic Extraction - Oil and Gas            0.000000e+00   
Unused Domestic Extraction - Non-metalic Minerals   0.000000e+00   
Unused Domestic Extraction - Iron Ore      

In [29]:
exiobase_pymrio.satellite.F

region                                                     AT               \
sector                                             Paddy rice        Wheat   
stressor                                                                     
Taxes less subsidies on products purchased: Total         0.0     6.053770   
Other net taxes on production                             0.0   -12.220461   
Compensation of employees; wages, salaries, & e...        0.0     1.065933   
Compensation of employees; wages, salaries, & e...        0.0    14.668017   
Compensation of employees; wages, salaries, & e...        0.0     2.336576   
...                                                       ...          ...   
Energy Carrier Net TMAR                                   0.0     0.000000   
Energy Carrier Net TOTH                                   0.0     0.000000   
Energy Carrier Net TRAI                                   0.0     0.000000   
Energy Carrier Net TROA                                   0.0  1366.420118   
Energy Carrier Net LOSS                                   0.0     0.000000   

region                                                                \
sector                                             Cereal grains nec   
stressor                                                               
Taxes less subsidies on products purchased: Total          11.889503   
Other net taxes on production                             -26.910668   
Compensation of employees; wages, salaries, & e...          2.166877   
Compensation of employees; wages, salaries, & e...         29.817821   
Compensation of employees; wages, salaries, & e...          4.749900   
...                                                              ...   
Energy Carrier Net TMAR                                     0.000000   
Energy Carrier Net TOTH                                     0.000000   
Energy Carrier Net TRAI                                     0.000000   
Energy Carrier Net TROA                                   233.037927   
Energy Carrier Net LOSS                                     0.000000   

region                                                                      \
sector                                             Vegetables, fruit, nuts   
stressor                                                                     
Taxes less subsidies on products purchased: Total                10.886212   
Other net taxes on production                                   -39.789720   
Compensation of employees; wages, salaries, & e...                3.665732   
Compensation of employees; wages, salaries, & e...               50.443163   
Compensation of employees; wages, salaries, & e...                8.035462   
...                                                                    ...   
Energy Carrier Net TMAR                                           0.000000   
Energy Carrier Net TOTH                                           0.000000   
Energy Carrier Net TRAI                                           0.000000   
Energy Carrier Net TROA                                         650.000000   
Energy Carrier Net LOSS                                           0.000000   

region                                                         \
sector                                              Oil seeds   
stressor                                                        
Taxes less subsidies on products purchased: Total    1.314695   
Other net taxes on production                       -5.832781   
Compensation of employees; wages, salaries, & e...   0.249154   
Compensation of employees; wages, salaries, & e...   3.428544   
Compensation of employees; wages, salaries, & e...   0.546158   
...                                                       ...   
Energy Carrier Net TMAR                              0.000000   
Energy Carrier Net TOTH                              0.000000   
Energy Carrier Net TRAI                              0.000000   
Energy Carrier Net TROA  